In [1]:
from ast import While
from sys import deactivate_stack_trampoline
import time
from fastapi import FastAPI, HTTPException
from fastapi import status
import psycopg2
from pydantic import BaseModel
import easyocr

In [2]:
app = FastAPI()
reader = easyocr.Reader(['en'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [3]:
while True:
    try:
        conn = psycopg2.connect(database="postgres", user="postgres.cidsbhqdqcupmmfgovdb", password="brainalyst@123", host="aws-0-ap-south-1.pooler.supabase.com", port="5432")
        cursor = conn.cursor()
        print("Connected to DB")
        break
    except Exception as error:
        print("Unable to connect to DB")
        time.sleep(2)


Connected to DB


In [4]:
user_id = None


In [5]:
def login(username:str,password:str):
    cursor.execute("SELECT id FROM customers WHERE username = %s AND password = %s",(username,password))
    user_id = cursor.fetchone()
    if not user_id:
        print("Wrong username or password")
    else:
        print("Login Successful")
        return user_id

In [6]:
while True:
    print("\nPlease Login")
    username = input("Enter username: ")
    password = input("Enter password: ")
    user_id = login(username=username,password=password)
    if user_id:
        break
while True:
    print("\nWelcome to the system, How can I help you?")
    print("1. Property Details")
    print("2. Add Property")
    print("3. Update Property")
    print("4. Scan property document")
    print("5. Log out")
    option = input("\nEnter your choice: ")
    if option == '1':
        cursor.execute("SELECT * FROM properties WHERE customer_id = %s",(user_id))
        print("\n")
        properties = cursor.fetchall()
        for  i in properties:
            print(f"Name: {i[1]}")
            print(f"Address: {i[2]}, {i[3]}, {i[4]}\n")
    elif option == '2':
        name = input("Enter property name: ")
        print("Enter property Address")
        street = input("Street: ")
        district = input("District: ")
        state = input("State: ")
        cursor.execute("INSERT INTO properties (property_name,street,district,state,customer_id) VALUES (%s,%s,%s,%s,%s) RETURNING property_id",(name,street,district,state,user_id))
        property_id = cursor.fetchone()
        print(property_id)
        cursor.execute("SELECT * FROM properties WHERE property_id = %s",(property_id))
        properties = cursor.fetchall()
        for  i in properties:
            print(f"Name: {i[1]}")
            print(f"Address: {i[2]}, {i[3]}, {i[4]}\n")
        conn.commit()
        print("Property added successfully")
    elif option == '3':
        cursor.execute("SELECT * FROM properties WHERE customer_id = %s",(user_id))
        properties = cursor.fetchall()
        print(properties)
        property_id = input("Enter property id to update: ")
        name = input("Enter property name: ")
        street = input("Street: ")
        district = input("District: ")
        state = input("State: ")
        property_id = cursor.execute("UPDATE properties SET name = %s, street = %s, district = %s, state = %s WHERE id = %s returning property_id",(name,street,district,state,property_id))
        conn.commit()
        print("Property updated successfully")
    elif option == '4':
        print("Scanning--------------")
        result= reader.readtext('prop_doc.png')
        info=[]
        for (bbox,text,prob) in result:
            info.append(text)
        property_name = info[1]
        street = info[3]
        district = info[5]
        state = info[7]
        cursor.execute("INSERT INTO properties (property_name,street,district,state,customer_id) VALUES (%s,%s,%s,%s,%s) RETURNING property_id",(property_name,street,district,state,user_id))
        property_id = cursor.fetchone()
        print(property_id)
        cursor.execute("SELECT * FROM properties WHERE property_id = %s",(property_id))
        properties = cursor.fetchall()
        for  i in properties:
            print(f"Name: {i[1]}")
            print(f"Address: {i[2]}, {i[3]}, {i[4]}\n")
        conn.commit()
        print("Property added successfully")
        info.clear()
    elif option == '5':
        print("Logged out success!!")
        break
    else:
        print("Invalid option")
    


Please Login


Enter username:  Abhi56
Enter password:  asdfqw


Login Successful

Welcome to the system, How can I help you?
1. Property Details
2. Add Property
3. Update Property
4. Scan property document
5. Log out



Enter your choice:  4


Scanning--------------
(11,)
Name: Orchid Inn
Address: Koramangala, Bangalore, Karnataka

Property added successfully

Welcome to the system, How can I help you?
1. Property Details
2. Add Property
3. Update Property
4. Scan property document
5. Log out



Enter your choice:  5


Logged out success!!
